In [1]:
import gym
import ale_py
import random
import numpy as np

from math_functions import log_stable
from math_functions import softmax
from math_functions import onehot
from math_functions import kl_div
from math_functions import obj_array,obj_array_zeros

from scipy.stats import dirichlet

# Agent functions
from ai_agent_planner import action_dist
from infer_state import infer_state

In [2]:
env = gym.make('Pong-v0', render_mode='rgb_array')
action_space=np.array([1,2])

In [3]:
env.observation_space.shape

(210, 160, 3)

In [4]:
import time

In [5]:
for i_episode in range(5):
    observation = env.reset()
    r=0
    for t in range(1000):
        env.render()    
        n='name'
        action = np.random.choice(action_space)
        observation, reward, done, info = env.step(action)
#         print(action)
        if(t==500):
            ss=observation
            env.env.ale.saveScreenPNG(b'test_image.png')
#         print(reward)
        r+=reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
#             print(r)
            break
        time.sleep(0.5)     
env.close()

C:\Users\aswin\anaconda3\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
C:\Users\aswin\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


KeyboardInterrupt: 

In [6]:
env.close()

In [8]:
s0=np.zeros(128)
sn=np.zeros(128)+255
factor=0.5
n_s=int((1/factor)*255)+1

In [9]:
states=np.zeros([n_s,128])
for i in range(n_s):
    states[i]=s0+factor*i

In [10]:
EPS_VAL = 1e-16 #negligibleconstant

In [11]:
#Number of states and number of controls/action
n_s=511
n_a=3
num_states=[n_s]
num_controls=[n_a]

num_obs=[n_s,3]
num_factors=len(num_states)
num_control_factors=len(num_controls)
num_modalities = len(num_obs) 

In [12]:
Blearned=obj_array(num_factors)
Alearned=obj_array(num_modalities)

Blearned[0]=np.zeros((n_s,n_s,n_a))

Alearned[0]=np.zeros((n_s,n_s))
Alearned[1]=np.zeros((3,n_s))

#prior preferences in terms of observations
C = obj_array_zeros(num_obs)
C[1]=[-1,-1,1]

#prior at t=0
D = obj_array(num_factors)
D[0]=np.zeros((num_states[0]))
prior=D[0]

#Sm_pars list
sm_pars=[100]
smpt=len(sm_pars)
trials=1
time_steps=1000
T=4

Alearned[0]=np.zeros((n_s,n_s))
a_prior=obj_array(num_modalities)
a_prior[0]=np.zeros((n_s,n_s))
a_prior[1]=np.zeros((3,n_s))

b_prior=np.zeros((n_s,n_s,n_a))
numS=num_states[0]
numA=num_controls[0]
numO1=num_obs[0]
numO2=num_obs[1]

In [13]:
episodes=np.zeros((smpt,trials,time_steps))

In [73]:
EPS_VAL = 1e-16 #negligibleconstant

In [80]:
for sp in range(smpt):
    print('Gamma',sp)
    
    for ii in range(trials):
        print('Trial',ii)
        epi=0
        tau=0

        #Learning Transition dynamics
        #Dirichlet distribution (Priors)
        b=b_prior+EPS_VAL #Hidden-states
        a1=a_prior[0]+EPS_VAL #Mod-1
        a2=a_prior[1]+EPS_VAL #Mod-2

        Blearned=obj_array(num_factors)
        Alearned=obj_array(num_modalities)
        Blearned[0]=np.zeros((n_s,n_s,n_a))
        Alearned[0]=np.zeros((n_s,n_s))
        Alearned[1]=np.zeros((3,n_s))

        for i in range(n_s):
            Alearned[0][:,i]=dirichlet.mean(a1[:,i])
            Alearned[1][:,i]=dirichlet.mean(a2[:,i])
            for j in range(n_a):
                Blearned[0][:,i,j]=dirichlet.mean(b[:,i,j])


        #Planning using available A,B,C,T
        Qactions=action_dist(Alearned,Blearned,C,T,sm_pars[sp])

        for ts in range(time_steps):
            print("ts=",ts)
            if(tau==0):
                #New episode-start
                prior=D[0]
                global state
                state=env.reset()
                obs_idx=[state,0]

            kingsmoves=[0,1,2]
            #Perception
            q_s=infer_state(prior,Alearned,obs_idx)

            a1+=np.kron(q_s,onehot(obs_idx[0],numO1).reshape((-1,1)))
            a2+=np.kron(q_s,onehot(obs_idx[1],numO2).reshape((-1,1)))

            action_dist_qs=Qactions[tau,:,:].dot(q_s)
            action=np.random.choice(kingsmoves,p=action_dist_qs)
            state, reward, done, info = env.step(action)
            obs_idx[0]=state
            obs_idx[1]=int(reward)
            print("reward=",reward)
            print(done)

            if(tau>1):
                b[:,:,ac_mo]+=Qactions[tau,ac_mo,:].dot(q_s_mo)*np.kron(q_s_mo,q_s.reshape((-1,1)))

            tau+=1
            print("tau=",tau)
            #End of planning trial conditions
            if(tau==T-1):
                tau=0              
                for i in range(numS):
                    Alearned[0][:,i]=dirichlet.mean(a1[:,i])
                    Alearned[1][:,i]=dirichlet.mean(a2[:,i])
                    for j in range(numA):
                        Blearned[0][:,i,j]=dirichlet.mean(b[:,i,j])
                #Replanning with new_evidence
                Qactions=action_dist(Alearned,Blearned,C,T,sm_pars[sp])

            if(reward==1):
                epi+=1
                tau=0
            #Updating episodes completed
            episodes[sp,ii,ts]=epi

#             Blearneddummy=np.zeros((numS,numS,numA))
#             Alearneddummy=np.zeros((numO1,numA))
#             Blearneddummy=np.array((Blearned[0]))
#             Alearneddummy=np.array((Alearned[0]))
#             for i in range(numS):
#                 obsenoisedev_t_1[sp,ii,ts]+=kl_div(A_true[0][:,i],Alearneddummy[:,i])
#                 for j in range(numA):
#                     modeldeviation_t_1[sp,ii,ts]+=kl_div(Btrue_1[:,i,j],Blearneddummy[:,i,j])

            #Setting-up priors for next time_step
            prior=Blearned[0][:,:,action].dot(q_s)
            q_s_mo=q_s
            ac_mo=action

Gamma 0
Trial 0
ts= 0
reward= 0.0
False
tau= 1
ts= 1
reward= 0.0
False
tau= 2
ts= 2
reward= 0.0
False
tau= 3


KeyboardInterrupt: 

In [ ]:
ts

In [ ]:
state

In [ ]:
reward

In [61]:
obs_idx[1]

0.0

In [78]:
state=env.reset()

In [79]:
state

array([192,   0,   0,   0, 110,  38,   0,   7,  71,   1,  60,  59,   0,
         0,   0,  62, 255,   0, 255, 253,   0,  22,   0,  24, 128,  32,
         1,  86, 247,  86, 247,  86, 247, 134, 243, 245, 243, 240, 240,
       242, 242,  32,  32,  64,  64,  64, 188,  65, 189,   0,  22, 109,
        37,  37,  60,   0,   0,   0,   0, 109, 109,  37,  37, 192, 192,
       192, 192,   1, 192, 202, 247, 202, 247, 202, 247, 202, 247,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,  54, 236, 242, 121, 240], dtype=uint8)